In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [431]:
from math import floor, ceil
from copy import deepcopy

class SnailPair:
    def __init__(self, 
                 left_val, 
                 right_val ,
                 nesting=0,
                 parent=None, 
                 is_left=False, 
                 is_right=False,
                 left_idx = float('inf'),
                 right_idx = float('inf'),
                ):
        
        if type(left_val) == SnailPair:
            left_val.update_parent(self)
            left_val.update_side('left')
            
        if type(right_val) == SnailPair:
            right_val.update_side('right')
            right_val.update_parent(self)
        
        self.left_val = left_val
        self.right_val = right_val
        
        self.parent = parent
        self.nesting = nesting
        
        self.is_left= is_left
        self.is_right= is_right
        
        self.left_idx = left_idx
        self.right_idx = right_idx
        
    def update_parent(self, parent):
        self.parent = parent
        
    def update_side(self, side):
        if side == 'left':
            self.is_left = True
            self.is_right = False
        if side == 'right':
            self.is_left = False
            self.is_right = True
            
        
    def __str__(self):
        def dive_for_string(snailnum):
            left =snailnum.left_val
            right = snailnum.right_val
            if type(left) == SnailPair and type(right) == int:
                return '['+dive_for_string(left)+f',{right:d}]'
            if type(left) == SnailPair and type(right) == SnailPair:
                return '['+dive_for_string(left)+',' +dive_for_string(right)+']'
            if type(left) == int and type(right) == SnailPair:
                return f'[{left},'+dive_for_string(right)+']'
            if  type(left) == int and type(right) == int:
                return f'[{left},{right}]'
        return dive_for_string(self)
    
    def reduce_number_single_step(self,snumber=None):
                
        def process_split(parent, number, right=False, left=False):
            if right:
                parent.right_val = SnailPair(int(floor(number/2)), 
                                             int(ceil(number/2)),
                                             nesting = parent.nesting+1,
                                             parent = parent,
                                             is_right = True, 
                                            )
            if left:
                parent.left_val = SnailPair(int(floor(number/2)), 
                                            int(ceil(number/2)),
                                            nesting = parent.nesting+1,
                                            parent = parent,
                                            is_left = True, 
                                            )
            
        def add_right_child(snailnum, number):
            while(type(snailnum.right_val)==SnailPair):
                snailnum = snailnum.right_val
            snailnum.right_val += number
            
            
        def add_left_child(snailnum, number):
            while(type(snailnum.left_val)==SnailPair):
                snailnum = snailnum.left_val
            snailnum.left_val += number
            
            
        def find_neighbor_left(snailnum):
            while snailnum.is_left:
                snailnum = snailnum.parent
            if snailnum.is_right:
                return snailnum
            else:
                return None
            
        def find_neighbor_right(snailnum):
            while snailnum.is_right:
                snailnum = snailnum.parent
            if snailnum.is_left:
                return snailnum
            else:
                return None
            
        def explode_snail_pair(snailnum):
            parent = snailnum.parent
            left_num = snailnum.left_val
            right_num = snailnum.right_val
            
            if snailnum.is_left:
                parent.left_val =0
            if snailnum.is_right:
                parent.right_val=0
                
            if snailnum.is_left:
                if type(parent.right_val) ==int:
                    parent.right_val += right_num
                else:
                    add_left_child(parent.right_val, right_num)
                now_right = find_neighbor_left(snailnum)
                if now_right is not None:
                    right_parent = now_right.parent
                    if right_parent is not None:
                        if type(right_parent.left_val) == int:
                            right_parent.left_val += left_num
                            
                        else:
                            add_right_child(right_parent.left_val, left_num)
                        
            if snailnum.is_right:
                if type(parent.left_val) ==int:
                    parent.left_val +=  left_num
                    
                else:
                    add_right_child(parent.left_val, left_num)
                    
                now_left = find_neighbor_right(snailnum)
                if now_left is not None:
                    left_parent = now_left.parent
                    if left_parent is not None:
                        if type(left_parent.right_val) == int:
                            left_parent.right_val += right_num
                        else:
                            add_left_child(left_parent.right_val, right_num)
                            
        def get_actions_by_line(snailnum ):
            splits=[]
            explosions=[]
            
            
            brackets = []
            numbers = []
            
            line = str(snailnum)
            
            idx=0
            while idx < len(line):
                s= line[idx]
                if s == '[':
                    brackets.append(s)
                    idx += 1
                elif s == ']':
                    nesting=len(brackets)
                    r = numbers.pop()
                    l = numbers.pop()
                    _ = brackets.pop()
                    new_snail_pair = SnailPair(left_val=l[0], right_val=r[0], nesting=nesting) 
                    numbers.append((new_snail_pair, l[1]))
                    
                    if new_snail_pair.nesting > 4:
#                        actions.append([0,l[1],explode_snail_pair, new_snail_pair])
                        explosions.append([explode_snail_pair, new_snail_pair])
                        
                    if type(l[0]) ==int and l[0] > 9 :
                       # actions.append([1,l[1],process_split, new_snail_pair, l[0], False, True ])
                        splits.append([process_split, new_snail_pair, l[0], False, True ])
                    if type(r[0])==int and r[0] > 9 :
                        #actions.append([1,r[1],process_split, new_snail_pair, r[0], True, False ])
                        splits.append([process_split, new_snail_pair, r[0], True, False ])
                    
                    idx += 1
                elif s == ',':
                    idx += 1
                    continue
                
                else:
                    a=1
                    while line[idx:idx+a].isdigit():
                        a+=1
                        
                    numbers.append((int(line[idx:idx+a-1]), idx))
                    idx += a-1
            return numbers[0][0], explosions, splits
       
        new_number = deepcopy(snumber) if snumber is not None else deepcopy(self)
        #new_number, actions = get_actions_by_line(new_number)
        #actions = sorted(actions, key=lambda act: act[0:2])
        #if len(actions) > 0:
        #    act = actions[0][2:]
        #    act[0](*act[1:])
        
        new_number, explosions, splits = get_actions_by_line(new_number)
        nexplosions = len(explosions)
        for exp in explosions:
            exp[0](*exp[1:])
        if nexplosions ==0 and len(splits) > 0:
            splits[0][0](*splits[0][1:])
            
        return new_number, nexplosions+len(splits)
    
    def reduce_number(self):
        orig_number = deepcopy(self)
        
        reduced, nactions = self.reduce_number_single_step(orig_number)
        while nactions >0 : 
            reduced, nactions =self.reduce_number_single_step(reduced)
        return reduced
        
    def magnitude(self):
        left =self.left_val
        right = self.right_val
        
        if type(left) == SnailPair and type(right) == int:
            return 3*left.magnitude()+2*right
        if type(left) == SnailPair and type(right) == SnailPair:
            return  3*left.magnitude()+2*right.magnitude()
        if type(left) == int and type(right) == SnailPair:
            return  3*left+2*right.magnitude()
        if  type(left) == int and type(right) == int:
            return  3*left+2*right
            

In [432]:
def read_line(line):
    brackets = []
    numbers = []
    
    idx=0
    while idx < len(line):
        s= line[idx]
        if s == '[':
            brackets.append(s)
            idx += 1
        elif s == ']':
            nesting=len(brackets)
            r = numbers.pop()
            l = numbers.pop()
            _ = brackets.pop()
            numbers.append(SnailPair(left_val=l, right_val=r, nesting=nesting))
            idx += 1
        elif s == ',':
            idx += 1
            continue
        
        else:
            a=1
            while line[idx:idx+a].isdigit():
                a+=1
                
            numbers.append(int(line[idx:idx+a-1]))
            idx += a-1
    return numbers[0]


def compute_the_lines(list_numbers):
    def add_string(l1,l2):
        return f'[{str(l1)},{str(l2)}]'
    
    prev_line = list_numbers[0]
    for new_line in list_numbers[1:]:
        addition_string = add_string(prev_line, new_line)
        prev_line = str(read_line(addition_string).reduce_number())
    
    #    print(prev_line+'\n\n')
    print(prev_line)
    return read_line(prev_line)
        
        

In [433]:
test_list = [
'[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
'[[[5,[2,8]],4],[5,[[9,9],0]]]',
'[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]',
'[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]',
'[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]',
'[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]',
'[[[[5,4],[7,7]],8],[[8,3],8]]',
'[[9,3],[[9,9],[6,[4,9]]]]',
'[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]',
'[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]',
]
#4140

In [434]:
test_list_2 = [
'[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]',
'[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]',
'[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]',
'[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]',
'[7,[5,[[3,8],[1,4]]]]',
'[[2,[2,2]],[8,[8,1]]]',
'[2,9]',
'[1,[[[9,3],9],[[9,0],[0,7]]]]',
'[[[5,[7,4]],7],1]',
'[[[[4,2],2],6],[8,7]]',
]


In [435]:
simple_list = [
'[1,1]',
'[2,2]',
'[3,3]',
'[4,4]',
'[5,5]',
'[6,6]',
]

In [436]:
first_step = '[[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]],[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]]'
str(read_line(first_step).reduce_number())
'[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]'

'[[[[4,0],[5,4]],[[7,7],[6,0]]],[[[5,6],[5,6]],[[7,9],[5,0]]]]'

'[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]'

In [437]:
str(compute_the_lines(simple_list).reduce_number())

[[[[5,0],[7,4]],[5,5]],[6,6]]


'[[[[5,0],[7,4]],[5,5]],[6,6]]'

In [438]:
compute_the_lines(test_list).magnitude()

[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]


4140

In [439]:
compute_the_lines(test_list_2).magnitude()

[[[[7,7],[7,7]],[[7,0],[8,8]]],[[[7,6],[5,6]],[8,7]]]


3503

In [440]:
reduce_test = {'[[[[[9,8],1],2],3],4]' : '[[[[0,9],2],3],4]',
               '[7,[6,[5,[4,[3,2]]]]]' : '[7,[6,[5,[7,0]]]]',
               '[[6,[5,[4,[3,2]]]],1]' : '[[6,[5,[7,0]]],3]',
               '[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]' : '[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]',
               '[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]' : '[[3,[2,[8,0]]],[9,[5,[7,0]]]]'}

In [441]:
for inp, exp in reduce_test.items():
    test = read_line(inp)
    print( exp == str(test.reduce_number_single_step()[0]))
    

True
True
True
False
True


In [442]:
reduce = read_line('[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]')
print(reduce.reduce_number())
print('[[[[0,7],4],[[7,8],[6,0]]],[8,1]]')

[[[[0,7],4],[[7,8],[6,0]]],[8,1]]
[[[[0,7],4],[[7,8],[6,0]]],[8,1]]


In [443]:
str(read_line('[[[0[15,9]],0],1]').reduce_number())

'[[[7,[8,0]],9],1]'

# Part One

In [444]:
production = []
with open('data/input_18.txt', 'r') as f:
    production = [ l.strip() for l in f.readlines()]

In [445]:
compute_the_lines(production).magnitude()

[[[[7,7],[7,0]],[[7,7],[8,8]]],[[[8,8],[8,8]],[[8,8],[8,7]]]]


4417

# Part Two

In [446]:
def compute_max_magnitude(list_numbers):
    def add_string(l1,l2):
        return f'[{str(l1)},{str(l2)}]'
    
    max_mag = 0
    for idx,line1 in enumerate(list_numbers[:-1]):
        for line2 in list_numbers[idx+1:]:
            combo1 = add_string(line1,line2)
            mag1 = read_line(combo1).reduce_number().magnitude()
            max_mag = max(mag1, max_mag)
            
            combo2 = add_string(line2,line1)
            mag2 = read_line(combo2).reduce_number().magnitude()
            max_mag = max(mag2, max_mag)
    return max_mag
        

In [448]:
%%time
compute_max_magnitude(production)

CPU times: user 27 s, sys: 34.9 ms, total: 27 s
Wall time: 27 s


4726